In [84]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

In [85]:
meta_input = "../data/input_meta.csv"
df=pd.read_csv(meta_input,low_memory=False)
search_zip_code = df["zip"][0]
search_job_title = df["job_title"][0]

search_zip_code, search_job_title

(95050, 'Data Scientist')

In [86]:
indeed_url=f"https://www.indeed.com/jobs?as_and={search_job_title}&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=&radius=50&l={search_zip_code}&fromage=3&limit=50&sort=&psf=advsrch"

In [87]:
executable_path = {"executable_path": "chromedriver"}
browser1 = Browser("chrome", **executable_path, headless=False)

In [88]:
# URL of page to be scraped
indeed_search_results = indeed_url 
browser1.visit(indeed_search_results)

In [89]:
try:
    browser1.click_link_by_id(id="prime-popover-close-button")
except:
    print("no popup")

In [90]:
#get the source code of the browser
html1 = browser1.html

In [91]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html1, 'html.parser')

In [92]:
# Examine the results, then determine element that contains sought info
# print(soup.prettify())

In [93]:
indeed_data={}
title=[]
company=[]
city=[]
state=[]
url=[]
description=[]

In [94]:
results = soup.find_all("div", class_="row result clickcard")
for i in range(len(results)):
    title.append(results[i].find('a', class_="turnstileLink").text)
    url.append(f'https://www.indeed.com{results[i].a["href"]}')
    
    company_name = results[i].find("span", class_="company").text.split("\n")
    
    for row in company_name:
        if row != " ":
            name = row
    
    company.append(name.lstrip())
    a,b = results[i].find("span", class_="location").text.split(",")
    city.append(a)
    state.append(b[1:3])


results = soup.find_all("div", class_="lastRow row result clickcard")
for i in range(len(results)):
    title.append(results[i].find('a', class_="turnstileLink").text)
    url.append(f'https://www.indeed.com{results[i].a["href"]}')
    
    company_name = results[i].find("span", class_="company").text.split("\n")
    
    for row in company_name:
        if row != " ":
            name = row
    
    company.append(name.lstrip())
    try:
        a,b = results[i].find("span", class_="location").text.split(",")
        city.append(a)
        state.append(b[1:3])
    except:
        city.append("San Jose")
        state.append("CA")
    
results = soup.find_all("div", class_="row sjlast result clickcard")
for i in range(len(results)):
    title.append(results[i].find('a', class_="turnstileLink").text)
    url.append(f'https://www.indeed.com{results[i].a["href"]}')
    
    company_name = results[i].find("span", class_="company").text.split("\n")
    
    for row in company_name:
        if row != " ":
            name = row
    
    company.append(name.lstrip())
    try:
        a,b = results[i].find("span", class_="location").text.split(",")
        city.append(a)
        state.append(b[1:3])
    except:
        city.append("San Jose")
        state.append("CA")    

In [95]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser3 = Browser('chrome', **executable_path, headless=True)

for link in url:
    job_desc_url = link 
    browser3.visit(job_desc_url)
    html3 = browser3.html
    soup3 = BeautifulSoup(html3, 'html.parser')
    try:
        job_desc = soup3.find("span",id="job_summary")
        description.append(job_desc.text)
    except:
        print(job_desc_url)
        job_desc = soup3.find("body")
        description.append(f"'{job_desc}'")


https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DhVAxkc_TxySVbUOs6bxWYWOfhmDTNcVTjFFBAY1FXZ_jbp8Q_DSbxEHI3oKBhlFyeWlu68uGMYtAI554qXEkHhqxMhyFhvvRor6-Zhil3TerMpZaO_nSbo4rw9Je3JC9zQoK76N5Vmm6X9wKdND-JOGYtKUtn0gMjCVSqALLR70rO3g4VSEUAEwsRtr8lHHZt8aiwYg7zMtBGbqzlBB_wv13qf7CO-VIF7FoOAR6LRpZ4GPruU-vpvKlxd8kaPwfChx3O212rT-0H43jlBMiowwtCSY7Q-uAvTi8lPoViVHUbjlDwjrNJkgR8zYsnmxkUCTkywj05v204sY__9YaL9g3D9Nwq5U-GwJiFs8DLVYKuf_jrP1Njl1EmH55ePYYDsvDHfZY4mRb-s4TPKQtH8afJ2r1s5gPbDqmEtvfpaLf9Bxfecez-gUMcbCWOWGC8gvk0ZwyMm3dG1mIGpIayHHfcXFpHjTDz09GL60KFhw==&vjs=3&p=3&sk=&fvj=0


In [110]:
print(f"Len of URL {len(url)}")
print(f"Len of company {len(company)}")
print(f"Len of title {len(title)}")
print(f"Len of state {len(state)}")
print(f"Len of city {len(city)}")
print(f"Len of description {len(description)}")

indeed_db = []
for i in range(len(url)):
        indeed_dict = {}
        indeed_dict["title"] = title[i]
        indeed_dict["company"] = company[i]
        indeed_dict["city"] = city[i]
        indeed_dict["state"] = state[i]
        indeed_dict["url"] = url[i]
        indeed_dict["description"] = description[i]
        indeed_db.append(indeed_dict)

Len of URL 55
Len of company 55
Len of title 55
Len of state 55
Len of city 55
Len of description 55


In [97]:
# job_desc_url = url[0] 
# browser3.visit(job_desc_url)
# html3 = browser3.html
# soup3 = BeautifulSoup(html3, 'html.parser')
# try:
#     job_desc = soup3.find("span",id="job_summary")
#     print(job_desc.text)
# except:
#     print(job_desc_url)
#     job_desc = soup3.find("body")
#     print(job_desc)

In [98]:
import datetime
str(datetime.datetime.now())

'2018-06-14 18:58:10.634406'

In [99]:
import platform
a = platform.platform()
if a[0:4] == "Wind":
    print("windows")
else:
    print(a[0:4])

windows


In [100]:
indeed = [
{
"title":"Lead Data Scientist - Machine Learning",
"company":"TARGET",
"city":"Sunnyvale",
"state":"CA",
"url":"https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb",
},
{"title":"Data Scientist",
"company":"Ebay",
"city":"San Jose",
"state":"CA",
"url":"https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb"
},
{"title":"Data Engineer",
"company":"EBay",
"city":"Sunnyvale",
"state":"CA",
"url":"https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb"
}
]

indeed[0]

{'city': 'Sunnyvale',
 'company': 'TARGET',
 'state': 'CA',
 'title': 'Lead Data Scientist - Machine Learning',
 'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb'}

In [101]:
glassdoor = [
{
"title":"Lead Data Scientist - Machine Learning",
"company":"TARGET",
"city":"Sunnyvale",
"state":"CA",
"url":"https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb",
},
{"title":"Engineer",
"company":"Applied",
"city":"Santa Clara",
"state":"CA",
"url":"https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb"
},
{"title":"Data Engineer",
"company":"EBay",
"city":"Sunnyvale",
"state":"CA",
"url":"https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb"
}
]

glassdoor[0]

{'city': 'Sunnyvale',
 'company': 'TARGET',
 'state': 'CA',
 'title': 'Lead Data Scientist - Machine Learning',
 'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb'}

In [102]:
import pandas as pd
df1 = pd.DataFrame.from_dict(glassdoor)
df1["city"]=df1["city"].str.upper()
df1["state"]=df1["state"].str.upper()
df1["company"]=df1["company"].str.upper()
df1["title"]=df1["title"].str.upper()

df2 = pd.DataFrame.from_dict(indeed)
df2["city"]=df1["city"].str.upper()
df2["state"]=df1["state"].str.upper()
df2["company"]=df1["company"].str.upper()
df2["title"]=df1["title"].str.upper()

In [103]:
df1

,city,company,state,title,url
0,SUNNYVALE,TARGET,CA,LEAD DATA SCIENTIST - MACHINE LEARNING,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,SANTA CLARA,APPLIED,CA,ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,SUNNYVALE,EBAY,CA,DATA ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


In [104]:
df2

,city,company,state,title,url
0,SUNNYVALE,TARGET,CA,LEAD DATA SCIENTIST - MACHINE LEARNING,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,SANTA CLARA,APPLIED,CA,ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,SUNNYVALE,EBAY,CA,DATA ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


In [105]:
result = pd.concat([df1,df2])
result

,city,company,state,title,url
0,SUNNYVALE,TARGET,CA,LEAD DATA SCIENTIST - MACHINE LEARNING,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,SANTA CLARA,APPLIED,CA,ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,SUNNYVALE,EBAY,CA,DATA ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
0,SUNNYVALE,TARGET,CA,LEAD DATA SCIENTIST - MACHINE LEARNING,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,SANTA CLARA,APPLIED,CA,ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,SUNNYVALE,EBAY,CA,DATA ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


In [106]:
result.drop_duplicates(subset=['city', 'company', 'title'], keep="last",inplace=True)

In [107]:
result

,city,company,state,title,url
0,SUNNYVALE,TARGET,CA,LEAD DATA SCIENTIST - MACHINE LEARNING,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,SANTA CLARA,APPLIED,CA,ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,SUNNYVALE,EBAY,CA,DATA ENGINEER,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...


In [108]:
result.to_dict('records')

[{'city': 'SUNNYVALE',
  'company': 'TARGET',
  'state': 'CA',
  'title': 'LEAD DATA SCIENTIST - MACHINE LEARNING',
  'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb'},
 {'city': 'SANTA CLARA',
  'company': 'APPLIED',
  'state': 'CA',
  'title': 'ENGINEER',
  'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb'},
 {'city': 'SUNNYVALE',
  'company': 'EBAY',
  'state': 'CA',
  'title': 'DATA ENGINEER',
  'url': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Bw1nCGA-im3c4lgntb'}]

In [121]:
t_range = pd.date_range("00:00", "23:50", freq="60min").time